#**État des lieux des collèges** 

CSV utilisés : 

* [Résultats détaillés au DNB](https://data.occitanie.education.gouv.fr/explore/dataset/fr-en-resultats-detailles-au-dnb/table/?sort=annee)

* [Label numérique collèges académie de Montpellier](https://data.occitanie.education.gouv.fr/explore/dataset/fr-en-occitanie-ac-montpellier-label-numerique-college/information/)

* [Indices de position sociale dans les collèges de France métropolitaine et DROM](https://data.education.gouv.fr/explore/dataset/fr-en-ips_colleges/information/?disjunctive.rentree_scolaire&disjunctive.academie&disjunctive.code_du_departement&disjunctive.departement&disjunctive.uai&disjunctive.code_insee_de_la_commune&disjunctive.nom_de_la_commune&disjunctive.secteur)

## Analyses des indicateurs de résultats des collèges en occitanie

### Imports des packages utiles

In [473]:
# importing necessary packages
import pandas as pd

In [474]:
# import des librairies pour les graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [475]:
# Import necessary libraries for image
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium # for maps
import plotly.graph_objects as go
!pip install -U kaleido
import kaleido
from IPython.display import Image
import io
from PIL import Image as ImagePIL

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### ETL du CSV Résultats détaillés au DNB

In [476]:
# importing the csv files
df = pd.read_csv("fr-en-resultats-detailles-au-dnb.csv", sep = ";")
df

,année,académie,code région,libellé région,série,inscrits,inscrits filles,présents,présents filles,admis,...,admis mention assez bien,admis mention bien,admis mention très bien,genre,âge,présence,admissibilité,admissibilité détails,Résultats,Effectifs
0,2011,MONTPELLIER,76,Occitanie,Serie college,21265,10994.0,21057.0,10900.0,19223.0,...,6113.0,4197.0,1808.0,Garçons,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_garcons,114
1,2011,MONTPELLIER,76,Occitanie,Serie college,21265,10994.0,21057.0,10900.0,19223.0,...,6113.0,4197.0,1808.0,Garçons,15 ans ou moins,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,1803
2,2011,MONTPELLIER,76,Occitanie,Serie professionnelle,514,221.0,483.0,211.0,429.0,...,138.0,41.0,2.0,Filles,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_filles,10
3,2011,MONTPELLIER,76,Occitanie,Serie professionnelle,514,221.0,483.0,211.0,429.0,...,138.0,41.0,2.0,Garçons,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_garcons,21
4,2011,MONTPELLIER,76,Occitanie,Serie professionnelle,514,221.0,483.0,211.0,429.0,...,138.0,41.0,2.0,Filles,15 ans ou moins,Présents,Non admis,e. Non admis,non_admis_filles,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,2021,TOULOUSE,76,Occitanie,Serie professionnelle,1479,558.0,1335.0,500.0,983.0,...,330.0,187.0,42.0,Garçons,16 ans,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,118
1724,2021,TOULOUSE,76,Occitanie,Serie generale,91,35.0,65.0,26.0,36.0,...,10.0,2.0,2.0,Filles,17 ans ou plus,Présents,Admis,b. Admis mention bien,admis mention bien filles,0
1725,2021,TOULOUSE,76,Occitanie,Serie generale,91,35.0,65.0,26.0,36.0,...,10.0,2.0,2.0,Garçons,17 ans ou plus,Non présents,Non présents,f. Non présents,non_presents_garcons,17
1726,2021,TOULOUSE,76,Occitanie,Serie professionnelle,164,56.0,135.0,49.0,91.0,...,27.0,15.0,7.0,Filles,17 ans ou plus,Présents,Admis,d. Admis sans mention,admis sans mention filles,17


In [477]:
# afficher les infos des colonnes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   année                     1728 non-null   int64  
 1   académie                  1728 non-null   object 
 2   code région               1728 non-null   int64  
 3   libellé région            1728 non-null   object 
 4   série                     1728 non-null   object 
 5   inscrits                  1728 non-null   int64  
 6   inscrits filles           1728 non-null   float64
 7   présents                  1728 non-null   float64
 8   présents filles           1728 non-null   float64
 9   admis                     1728 non-null   float64
 10  admis filles              1728 non-null   float64
 11  admis sans mention        1728 non-null   float64
 12  admis mention assez bien  1728 non-null   float64
 13  admis mention bien        1728 non-null   float64
 14  admis me

In [478]:
# fonction pour transformer le type des colonnes d'un df 
def formatterCol(dataframe, col, na, type):
  dataframe[col] = dataframe[col].fillna(na).astype(type)
  return dataframe


In [479]:
# transformer le type des colonnes float en int
# je choisis de fill des NaN par 0 puisque ça n'impactera pas les analyses
# inscrits filles
df = formatterCol(df, "inscrits filles", 0, int)
# présents 
df = formatterCol(df, "présents", 0, int)
# présents filles
df = formatterCol(df, "présents filles", 0, int)
# admis 
df = formatterCol(df, "admis", 0, int)
# admis filles
df = formatterCol(df, "admis filles", 0, int)
# admis sans mention
df = formatterCol(df, "admis sans mention", 0, int)
# admis mention assez bien
df = formatterCol(df, "admis mention assez bien", 0, int)
# admis mention bien
df = formatterCol(df, "admis mention bien", 0, int)
# admis mention très bien
df = formatterCol(df, "admis mention très bien", 0, int)

In [480]:
# infos des colonnes post transformation
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   année                     1728 non-null   int64 
 1   académie                  1728 non-null   object
 2   code région               1728 non-null   int64 
 3   libellé région            1728 non-null   object
 4   série                     1728 non-null   object
 5   inscrits                  1728 non-null   int64 
 6   inscrits filles           1728 non-null   int64 
 7   présents                  1728 non-null   int64 
 8   présents filles           1728 non-null   int64 
 9   admis                     1728 non-null   int64 
 10  admis filles              1728 non-null   int64 
 11  admis sans mention        1728 non-null   int64 
 12  admis mention assez bien  1728 non-null   int64 
 13  admis mention bien        1728 non-null   int64 
 14  admis mention très bien 

In [481]:
# drop des colonnes pas très utiles 
# drop code région (76 partout)
df = df.drop(columns = "code région")
# drop libellé région (Occitanie partout)
df = df.drop(columns = "libellé région")
# vérifications
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   année                     1728 non-null   int64 
 1   académie                  1728 non-null   object
 2   série                     1728 non-null   object
 3   inscrits                  1728 non-null   int64 
 4   inscrits filles           1728 non-null   int64 
 5   présents                  1728 non-null   int64 
 6   présents filles           1728 non-null   int64 
 7   admis                     1728 non-null   int64 
 8   admis filles              1728 non-null   int64 
 9   admis sans mention        1728 non-null   int64 
 10  admis mention assez bien  1728 non-null   int64 
 11  admis mention bien        1728 non-null   int64 
 12  admis mention très bien   1728 non-null   int64 
 13  genre                     1728 non-null   object
 14  âge                     

,année,académie,série,inscrits,inscrits filles,présents,présents filles,admis,admis filles,admis sans mention,admis mention assez bien,admis mention bien,admis mention très bien,genre,âge,présence,admissibilité,admissibilité détails,Résultats,Effectifs
0,2011,MONTPELLIER,Serie college,21265,10994,21057,10900,19223,10088,7105,6113,4197,1808,Garçons,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_garcons,114
1,2011,MONTPELLIER,Serie college,21265,10994,21057,10900,19223,10088,7105,6113,4197,1808,Garçons,15 ans ou moins,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,1803
2,2011,MONTPELLIER,Serie professionnelle,514,221,483,211,429,193,248,138,41,2,Filles,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_filles,10
3,2011,MONTPELLIER,Serie professionnelle,514,221,483,211,429,193,248,138,41,2,Garçons,15 ans ou moins,Non présents,Non présents,f. Non présents,non_presents_garcons,21
4,2011,MONTPELLIER,Serie professionnelle,514,221,483,211,429,193,248,138,41,2,Filles,15 ans ou moins,Présents,Non admis,e. Non admis,non_admis_filles,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,2021,TOULOUSE,Serie professionnelle,1479,558,1335,500,983,370,424,330,187,42,Garçons,16 ans,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,118
1724,2021,TOULOUSE,Serie generale,91,35,65,26,36,16,22,10,2,2,Filles,17 ans ou plus,Présents,Admis,b. Admis mention bien,admis mention bien filles,0
1725,2021,TOULOUSE,Serie generale,91,35,65,26,36,16,22,10,2,2,Garçons,17 ans ou plus,Non présents,Non présents,f. Non présents,non_presents_garcons,17
1726,2021,TOULOUSE,Serie professionnelle,164,56,135,49,91,33,42,27,15,7,Filles,17 ans ou plus,Présents,Admis,d. Admis sans mention,admis sans mention filles,17


In [482]:
# Je veux connaître le nombre d'admis par année, région et série
# Je vais retirer dupliquer le dataframe et prendre que les colonnes utiles pour cette analyse
# duplication df
df2 = df[['année', 'académie', 'série', 'admis', 'inscrits']]
# retire les duplicates de résultats
df2 = df2.drop_duplicates(subset=['académie', 'inscrits'], keep='last')
# Trie la colonne année par année (ascendant : 2011 à 2021)
df2 = df2.sort_values(by="année")
# Transforme la colonne en string afin de pouvoir l'utiliser pour les graphiques
df2["année"] = df2["année"].apply(str)
# group by la région, l'année et la série afin d'avoir le nombre d'admis
result = df2.groupby(['année','académie', 'série', 'admis'])['admis'].count()
# print(result)
result


année  académie     série                  admis
2011   MONTPELLIER  Serie college          220      1
                                           2992     1
                                           19223    1
                    Serie professionnelle  180      1
                                           429      1
                                                   ..
2021   TOULOUSE     Serie generale         1250     1
                                           27517    1
                    Serie professionnelle  91       1
                                           983      1
                                           1679     1
Name: admis, Length: 143, dtype: int64

In [483]:
import re
aNegliger = re.compile("Serie college",re.I)
subst = "Serie generale"
def apply_reg_serie_college(str_):
    '''Fonction pour remplacer les formations de "Serie college" par "serie generale"     
        - paramètres : 
            - str_ : le nom de la colonne dans laquelle la fonction va chercher la pattern à nettoyé
        - Return la chaine de caractère nettoyé pour la colonne '''
    if pd.notna(str_):
        return re.sub(aNegliger, subst, str(str_))
    else:
        return str_

# copie du df2 pour appliquer les modifications sur la colonne 
df2_filtre = df2   
df2_filtre['série'] = df2['série'].apply(apply_reg_serie_college)



aNegliger2 = re.compile("Serie technologique")
def apply_reg_serie_techno(str_):
    '''Fonction pour remplacer les formations de "Serie technologique" par "Serie generale"     
        - paramètres : 
            - str_ : le nom de la colonne dans laquelle la fonction va chercher la pattern à nettoyé
        - Return la chaine de caractère nettoyé pour la colonne '''
    if pd.notna(str_):
        return re.sub(aNegliger2, subst, str(str_))
    else:
        return str_

# copie du df2 pour appliquer les modifications sur la colonne 
df2_filtre['série'] = df2['série'].apply(apply_reg_serie_techno)

df2_filtre

,année,académie,série,admis,inscrits
1394,2011,MONTPELLIER,Serie generale,402,466
1431,2011,TOULOUSE,Serie generale,193,314
1392,2011,MONTPELLIER,Serie professionnelle,429,514
1389,2011,MONTPELLIER,Serie generale,19223,21265
1401,2011,MONTPELLIER,Serie generale,713,994
...,...,...,...,...,...
1698,2021,MONTPELLIER,Serie professionnelle,1505,1892
1696,2021,MONTPELLIER,Serie generale,25305,28530
1725,2021,TOULOUSE,Serie generale,36,91
1709,2021,MONTPELLIER,Serie generale,41,132


### Analyses des résultats des collèges pour chaque académie

#### **1 - Montpellier**


**Nombre d'admis en fonction de l'année pour l'académie de Montpellier :**

In [484]:
# Division par académie
# Montpellier
df2M = df2[df2['académie'] == 'MONTPELLIER']
# Total admis pour toutes les séries
fig1 = px.histogram(df2M, x="année", y="admis", color="année")
fig1.update_layout(title_text="Distribution du nombre d'admis en fonction de l'année pour l'académie de Montpellier")
fig1.update_yaxes(title_text="Nombre d'admis")
fig1.update_xaxes(title_text="Année")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux voir que le nombre d'élèves admis **augmente chaque année**.
On remarque également **un pic à l'année 2020** qui pourrait être expliqué par l'allègement des examens dûs aux difficultés liés à la covid

---
**Nombre total d'inscrits en fonction de l'année pour l'académie de Montpellier :**

In [485]:
# Total d'inscrits pour toutes les séries
fig2 = px.histogram(df2M, x="année", y="inscrits", color="année")
fig2.update_layout(title_text="Distribution du nombre d'inscrits en fonction de l'année pour l'académie de Montpellier")
fig2.update_yaxes(title_text="Nombre d'inscrits")
fig2.update_xaxes(title_text="Année")
#fig2.show()

image = fig2.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'admis en fonction de l'année et de la série pour l'académie de Montpellier :**

In [486]:
# Montpellier : nb admis en fct de l'année et série
fig3 = px.histogram(df2M, x="année", y="admis", color="série", facet_col="série")
fig3.update_layout(title_text="Distribution du nombre d'admis en fonction de l'année et de la série pour l'académie de Montpellier")
fig3.update_yaxes(title_text="Nombre d'admis")
fig3.update_xaxes(title_text="Année")
# fig3.show()

image = fig3.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On constate que le nombre d'élèves admis en série générale est **nettement supérieur** aux élèves en série professionelle. 
Il n'y a pas autant de lycée de chaque série.

---
**Nombre d'inscrits en fonction de l'année et de la série pour l'académie de Montpellier :**

In [487]:
# Montpellier : nb d'inscrits en fct de l'année et série
fig4 = px.histogram(df2M, x="année", y="inscrits", color="série", facet_col="série")
fig4.update_layout(title_text="Distribution du nombre d'inscrits en fonction de l'année et de la série pour l'académie de Montpellier")
fig4.update_yaxes(title_text="Nombre d'inscrits")
fig4.update_xaxes(title_text="Année")
# fig4.show()

image = fig4.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux clairement voir **l'évolution du nombre d'élèves inscrits** par rapport aux **admis au fil des années**, suivant leur **série**.

On peux voir que pour l'académie de Montpellier, les élèves en **série générale** : 

*   le nombre **d'admis** en **2011** est de **23 671**
*   le nombre **d'admis** en **2021** est de **26 669**
*   le nombre **d'inscrits** en **2011** est de **29 084**
*   le nombre **d'inscrits** en **2021** est de **31 050**

Et pour les élèves en **série professionnelle** :

*  le nombre **d'admis** en **2011** est de **1469**
*  le nombre **d'admis** en **2021** est de **2279**
*  le nombre **d'inscrits** en **2011** est de **2299**
*  le nombre **d'inscrits** en **2021** est de **3036**

Au total : 

*  le nombre **d'admis** en **2011** est de **25 140**
*  le nombre **d'admis** en **2021** est de **28 948**
*  le nombre **d'inscrits** en **2011** est de **31 383**
*  le nombre **d'inscrits** en **2021** est de **34 086**

In [488]:
# ratio admis série générale
rasg = (26669 - 23671)/23671
print(rasg)
# ratio inscrits série générale
risg = (31050 - 29084)/29084
print(risg)
# ratio admis série professionnelle
rasp = (2279 - 1469)/1469
print(rasp)
# ratio inscrits série professionnelle
risp = (3036 - 2299)/2299
print(risp)
# ratio admis total
rat = (28948 - 25140)/25140
print(rat)
# ratio inscrit total
rit = (34086 - 31383)/31383
print(rit)

0.12665286637657894
0.06759730435978545
0.5513955071477196
0.32057416267942584
0.15147175815433572
0.08612943313258771


On constate que pour l'académie de Montpellier, les élèves en **série générale**, en **10 ans** (2011 à 2021) : 

*   le nombre d'admis a augmenté de **12,66%**
*   le nombre d'inscrits a augmenté de **6,76%** 

Et pour les élèves en **série professionnelle** :

*  le nombre d'admis a augmenté de **55,14%** 
*  le nombre d'inscrits a augmenté de **32,06%**

Enfin, au total on a : 

*  le nombre d'admis a augmenté de **15,15%**
*  le nombre d'inscrits a augmenté de **8,61%**



Etant donné que **le nombre d'admis augmente énormément par rapport au nombre d'inscrits**, on en déduit que le niveau global des collégiens **s'est amélioré en 10 ans**.

---
**Nombre d'inscrits en séries générales et technologiques à Montpellier en 2011**

In [489]:
# ratio inscrits série générale / série technologique pour 2011
asgvals = [1469, 23671]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'inscrits en séries générales et technologiques pour l'académie de Montpellier en 2011")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

Le nombre d'inscrits en série générale est **largement plus élevé** par rapport aux inscrits en série technologique en 2011.

---
**Nombre d'inscrits en séries générales et technolgiques à Montpellier en 2021**

In [490]:
# ratio inscrits série générale / série technologique pour 2021
asgvals = [2279, 26669]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'inscrits en séries générales et technologiques pour l'académie de Montpellier en 2021")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

Il y'a une **plus grande proportion d'élèves inscrits en série technologique en 2021 qu'en 2011** même si elle reste plutôt faible.

---
**Nombre d'admis en séries générales et technologiques à Montpellier en 2011**

In [491]:
 # ratio admis série générale / série technologique pour 2011
asgvals = [2299, 29084]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'admis en séries générales et technologiques pour l'académie de Montpellier en 2011")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'admis en séries générales et technologiques à Montpellier en 2021**

In [492]:
# ratio inscrits série générale / série technologique pour 2021
asgvals = [3036, 31050]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'admis en séries générales et technologiques pour l'académie de Montpellier en 2021")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

Même remarques pour les admis

#### **2 - Toulouse**

**Nombre d'admis en fonction de l'année au collège à Toulouse**

In [493]:
# Division par académie
# Toulouse
df2T = df2[df2['académie'] == 'TOULOUSE']
# Total admis pour toutes les séries
fig1 = px.histogram(df2T, x="année", y="admis", color="année")
fig1.update_layout(title_text="Distribution du nombre d'admis en fonction de l'année pour les collèges de l'académie de Toulouse")
fig1.update_yaxes(title_text="Nombre d'admis")
fig1.update_xaxes(title_text="Année")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

Même remarque que pour les résultats de l'académie de Montpellier.

On peux voir que le nombre d'élèves admis **augmente chaque année**.
On remarque également **un pic à l'année 2020** qui pourrait être expliqué par l'allègement des examens dûs aux difficultés liés à la covid.

---
**Nombre total d'inscrits en fonction de l'année au collège à Toulouse**

In [494]:
# Total d'inscrits pour toutes les séries
fig2 = px.histogram(df2T, x="année", y="inscrits", color="année")
fig2.update_layout(title_text="Distribution du nombre d'inscrits en fonction de l'année pour les collèges de l'académie de Toulouse")
fig2.update_yaxes(title_text="Nombre d'inscrits")
fig2.update_xaxes(title_text="Année")
# fig2.show()

image = fig2.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'admis en fonction de l'année et de la série à Toulouse**

In [495]:
# Toulouse : nb admis en fct de l'année et série
fig3 = px.histogram(df2T, x="année", y="admis", color="série", facet_col="série")
fig3.update_layout(title_text="Distribution du nombre d'admis en fonction de l'année et de la série pour les collèges de l'académie de Toulouse")
fig3.update_yaxes(title_text="Nombre d'admis")
fig3.update_xaxes(title_text="Année")
# fig3.show()

image = fig3.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'inscrits en fonction de l'année et de la série à Toulouse**

In [496]:
# Toulouse : nb d'inscrits en fct de l'année et série
fig4 = px.histogram(df2T, x="année", y="inscrits", color="série", facet_col="série")
fig4.update_layout(title_text="Distribution du nombre d'inscrits en fonction de l'année et de la série pour les collèges de l'académie de Toulouse")
fig4.update_yaxes(title_text="Nombre d'inscrits")
fig4.update_xaxes(title_text="Année")
# fig4.show()

image = fig4.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux voir que pour l'académie de Toulouse, pour les élèves **en série générale** : 

*   le nombre d'admis en **2011** est de **26 119**
*   le nombre d'admis en **2021** est de **28 803**
*   le nombre d'inscrits en **2011** est de **30 865**
*   le nombre d'inscrits en **2021** est de **32 345**

Et pour les élèves en **série professionnelle** :

*  le nombre d'admis en **2011** est de **803**
*  le nombre d'admis en **2021** est de **2753**
*  le nombre d'inscrits en **2011** est de **1555**
*  le nombre d'inscrits en **2021** est de **3783**

Au **total** : 

*  le nombre d'admis en **2011** est de **26 949**
*  le nombre d'admis en **2021** est de **31 556**
*  le nombre d'inscrits en **2011** est de **32 410**
*  le nombre d'inscrits en **2021** est de **36 128**

In [497]:
# ratio admis série générale
rasg = (28803 - 26119)/26119
print(rasg)
# ratio inscrits série générale
risg = (32345 - 30864)/30864
print(risg)
# ratio admis série professionnelle
rasp = (2753 - 803)/803
print(rasp)
# ratio inscrits série professionnelle
risp = (3783 - 1555)/1555
print(risp)
# ratio admis total
rat = (31556 - 26949)/26949
print(rat)
# ratio inscrit total
rit = (36128 - 32410)/32410
print(rit)

0.10276044258968567
0.047984707102125455
2.428393524283935
1.432797427652733
0.17095253998293072
0.11471767972847886


On constate que pour l'académie de Montpellier, les élèves en **série générale**, en **10 ans** (2011 à 2021) : 

*   le nombre d'admis a augmenté de **10,27%**
*   le nombre d'inscrits a augmenté de **4,79%**

Et pour les élèves en **série professionnelle** :

*  le nombre d'admis a augmenté de **242,8%** 
*  le nombre d'inscrits a augmenté de **143,2%**

Enfin, au **total** on a : 

*  le nombre d'admis a augmenté de **17,09%**
*  le nombre d'inscrits a augmenté de **11,47%**


Même constat que pour Montpellier : **le nombre d'admis augmente énormément par rapport au nombre d'inscrits**, on en déduit que **le niveau global des collégiens s'est amélioré en 10 ans**.


---
**Nombre d'inscrits en séries générales et technologiques à Toulouse en 2011**

In [498]:
# ratio inscrits série générale / série technologique pour 2011
asgvals = [803, 26119]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'inscrits en séries générales et technologiques pour les collèges de l'académie de Toulouse en 2011")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'inscrits en séries générales et technologiques à Toulouse en 2021**

In [499]:
# ratio inscrits série générale / série technologique pour 2021
asgvals = [2753, 28803]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'inscrits en séries générales et technologiques pour les collèges de l'académie de Toulouse en 2021")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux voir que la proportion d'élèves inscrits en série technologique a **nettement augmenté depuis 2011**. 
On est passés de **2,98%** à **8,72%** ce qui est largement supérieur à l'évolution de l'académie de Montpellier

---
**Nombre d'admis en séries générales et technologiques à Toulouse en 2011**

In [500]:
# ratio admis série générale / série technologique pour 2011
asgvals = [1555, 30865]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'admis en séries générales et technologiques pour les collèges de l'académie de Toulouse en 2011")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

---
**Nombre d'admis en séries générales et technologiques à Toulouse en 2021**

In [501]:
# ratio admis série générale / série technologique pour 2021
asgvals = [3783, 32345]
asglabels = ['Série technologique', 'Série générale']
fg1 = px.pie(values=asgvals, names= asglabels)
fg1.update_layout(title_text="Distribution du nombre d'admis en séries générales et technologiques pour les collèges de l'académie de Toulouse en 2021")
# fg1.show()

image = fg1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

Même remarque pour les admissions : 

On peux voir que la proportion d'élèves admis en série technologique a nettement **augmenté depuis 2011**. 
On est passés de **4,8%** à **10,5%** ce qui est largement supérieur à l'évolution de l'académie de Montpellier.

### ETL du CSV Label numérique collège académie de Montpellier

In [502]:
# importing the csv files
dflabels = pd.read_csv("fr-en-occitanie-ac-montpellier-label-numerique-college.csv", sep = ";")
dflabels

,Année,RNE,Nom établissement,Localité,Département,Label,Position
0,2017,0110686L,Collège Joseph Anglade,Lézignan-Corbières,11,2,"43.196809464, 2.76527043622"
1,2017,0110870L,Collège Antoine Pons,Chalabre,11,1,"42.9851089045, 2.00497993951"
2,2017,0111051H,Collège Rosa Parks,Lézignan-Corbières,11,2,"43.184518864, 2.7595799942"
3,2017,0300031W,Collège George Ville,Pont-Saint-Esprit,30,2,"44.2562769196, 4.64588054318"
4,2017,0301010K,Collège Jules Vallès,Nîmes,30,3,"43.8588460926, 4.39110910066"
...,...,...,...,...,...,...,...
1003,2022,0480011B,Collège André Chamson,Meyrueis,48,3,"44.1799498461, 3.43142941984"
1004,2022,0480016G,Collège du Trenze,Vialas,48,3,"44.3331989353, 3.89597048145"
1005,2022,0660431K,Collège Le Riberal,Saint-Estève,66,3,"42.7086982816, 2.83931047208"
1006,2022,0660634F,Collège Climatique et Sportif,Font-Romeu-Odeillo-Via,66,2,"42.5150970307, 2.03995955126"


In [503]:
# afficher les infos des colonnes
dflabels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Année              1008 non-null   int64 
 1   RNE                1008 non-null   object
 2   Nom établissement  1008 non-null   object
 3   Localité           1008 non-null   object
 4   Département        1008 non-null   int64 
 5   Label              1008 non-null   int64 
 6   Position           1008 non-null   object
dtypes: int64(3), object(4)
memory usage: 55.2+ KB


In [523]:
# Trie la colonne Label par labels (ascendant : de label 1 à label 3)
dflabels = dflabels.sort_values(by="Label")
dflabels = dflabels.sort_values(by="Année")
# dflabels.info()

In [524]:
# Transforme la colonne en string afin de pouvoir l'utiliser pour les graphiques
dflabels["Label"] = dflabels["Label"].apply(str)
dflabels["Département"] = dflabels["Département"].apply(str)

# copie temporaire pour les graphiques :
# dflabelsgraph = dflabels

dflabels["Année"] = dflabels["Année"].apply(str)

# dflabels.info()

### Analyses des labels pour les collèges

**Distribution du nombre de labels en fonction de l'année pour les collèges de l'académie de Montpellier**

In [526]:
# Nombre de labels en fct de l'année
fig1 = px.histogram(dflabels, x="Année", y="Label", color="Année", facet_col="Label")
fig1.update_layout(title_text="Distribution du nombre de labels en fonction de l'année pour les collèges de l'académie de Montpellier")
fig1.update_yaxes(title_text="Nombre de labels")
fig1.update_xaxes(title_text="Année")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On remarque que le nombre de collèges label **1** diminue au fil des ans tandis que les collèges labels **2** et **3** ne fait qu'augmenter.

---
**Distribution du nombre de labels en fonction de l'année pour les collèges de l'académie de Montpellier**

In [507]:
# Nombre de labels en fct de l'année
fig1 = px.histogram(dflabels, x="Année", y="Label", color="Année", facet_col="Label")
fig1.update_layout(title_text="Distribution du nombre de labels en fonction de l'année pour les collèges de l'académie de Montpellier")
fig1.update_yaxes(title_text="Nombre de labels")
fig1.update_xaxes(title_text="Année")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

In [508]:
# Total admis pour toutes les séries
fig1 = px.histogram(dflabels, x="Département", y="Label", color="Département", facet_col="Label")
fig1.update_layout(title_text="Distribution du nombre de labels en fonction du département pour les collèges de l'académie de Montpellier")
fig1.update_yaxes(title_text="Nombre de labels")
fig1.update_xaxes(title_text="Département")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux voir que les départements ayants le plus de collèges labéllisés **2** et **3** sont les départements de **l'Hérault (34), Gard (30) et Pyrénées-Orientales (66)**

In [509]:
# Compte du nombre de collèges dont le RNE est unique ayant un label de 1
nblabels1 = dflabels.query("Label == '1'")["RNE"].nunique()
# Compte du nombre de collèges dont le RNE est unique ayant un label de 2
nblabels2 = dflabels.query("Label == '2'")["RNE"].nunique()
# Compte du nombre de collèges dont le RNE est unique ayant un label de 3
nblabels3 = dflabels.query("Label == '3'")["RNE"].nunique()
labelvals = [nblabels1, nblabels2, nblabels3]
labelnames = ['Label 1', 'Label 2', 'Label 3']
fgl1 = px.pie(values=labelvals, names=labelnames)
fgl1.update_layout(title_text="Distribution du nombre de labels pour les collèges de l'académie de Montpellier")
# fgl1.show()

image = fgl1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux constater que il y'a plus de collèges labéllisés **3** que de collèges labéllisés **1** et **2**.

C'est un résultat assez prédictible étant donné que pour qu'un collège soit labellisé 3, il faut que ce collège réunisse **beaucoup de critères** en terme **d'infrastructure**, **d'équipements** et autres dans le domaine du numérique.

### ETL du CSV Indices de position sociale dans les collèges de France métropolitaine et DROM

In [510]:
# importing the csv files
dfIPS = pd.read_csv("fr-en-ips_colleges.csv", sep = ";")
dfIPS

,Rentrée scolaire,Académie,Code du département,Département,UAI,Nom de l'établissment,Code INSEE de la commune,Nom de la commune,Secteur,IPS
0,2021-2022,LYON,001,AIN,0010039M,COLLEGE DE L ALBARINE,01384,SAINT RAMBERT EN BUGEY,public,98.5
1,2021-2022,LYON,001,AIN,0010084L,COLLEGE PRIVE SAINT CHARLES,01093,CHATILLON SUR CHALARONNE,privé sous contrat,114.6
2,2021-2022,LYON,001,AIN,0010794H,COLLEGE DU VALROMEY,01022,ARTEMARE,public,95.6
3,2021-2022,LYON,001,AIN,0010796K,COLLEGE EMILE CIZAIN,01262,MONTLUEL,public,115.2
4,2021-2022,LYON,001,AIN,0010820L,COLLEGE GEORGE SAND,01306,PONT DE VEYLE,public,104.2
...,...,...,...,...,...,...,...,...,...,...
6962,2021-2022,LA REUNION,974,LA REUNION,9741649Z,COLLEGE PRIVE CATHOLIQUE LEVAVASSEUR,97418,SAINTE MARIE,privé sous contrat,133.2
6963,2021-2022,MAYOTTE,976,MAYOTTE,9760042S,COLLEGE MARCEL HENRY,97606,CHIRONGUI,public,72.2
6964,2021-2022,MAYOTTE,976,MAYOTTE,9760180S,COLLEGE MARIAMA SALIM DE SADA,97616,SADA,public,82.1
6965,2021-2022,MAYOTTE,976,MAYOTTE,9760219J,COLLEGE DE M GOMBANI,97611,MAMOUDZOU,public,74.1


In [511]:
# afficher les infos des colonnes
dfIPS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6967 entries, 0 to 6966
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rentrée scolaire          6967 non-null   object 
 1   Académie                  6967 non-null   object 
 2   Code du département       6967 non-null   object 
 3   Département               6967 non-null   object 
 4   UAI                       6967 non-null   object 
 5   Nom de l'établissment     6967 non-null   object 
 6   Code INSEE de la commune  6967 non-null   object 
 7   Nom de la commune         6967 non-null   object 
 8   Secteur                   6967 non-null   object 
 9   IPS                       6962 non-null   float64
dtypes: float64(1), object(9)
memory usage: 544.4+ KB


In [512]:
# Prendre uniquement les collèges dont l'académie est Montpellier
dfIPSM = dfIPS[dfIPS["Académie"] == 'MONTPELLIER']
print(dfIPS)

# les collèges montpellier et toulouse
dfIPST = dfIPS[dfIPS["Académie"] == 'TOULOUSE']
dfIPSALL = pd.concat([dfIPSM,dfIPST])
print(dfIPSALL)

     Rentrée scolaire    Académie Code du département Département       UAI  \
0           2021-2022        LYON                 001         AIN  0010039M   
1           2021-2022        LYON                 001         AIN  0010084L   
2           2021-2022        LYON                 001         AIN  0010794H   
3           2021-2022        LYON                 001         AIN  0010796K   
4           2021-2022        LYON                 001         AIN  0010820L   
...               ...         ...                 ...         ...       ...   
6962        2021-2022  LA REUNION                 974  LA REUNION  9741649Z   
6963        2021-2022     MAYOTTE                 976     MAYOTTE  9760042S   
6964        2021-2022     MAYOTTE                 976     MAYOTTE  9760180S   
6965        2021-2022     MAYOTTE                 976     MAYOTTE  9760219J   
6966        2021-2022     MAYOTTE                 976     MAYOTTE  9760379H   

                     Nom de l'établissment Code INS

In [513]:
# Transforme la colonne en string afin de pouvoir l'utiliser pour les graphiques
# dfIPS["Code du département"] = dflabels["Code du département"].apply(str)

In [ ]:
### Analyses des IPS pour les collèges

**Distribution de l'IPS moyen en fonction du département pour l'académie de Montpellier**

In [514]:
# IPS moyen en fct du département
fig1 = px.histogram(dfIPSM, x="Département", y="IPS", color="Département", histfunc="avg", )
fig1.update_layout(title_text="Distribution de l'IPS moyen en fonction du département pour les collèges de l'académie de Montpellier")
fig1.update_yaxes(title_text="IPS")
fig1.update_xaxes(title_text="Département")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On peux voir que l'IPS moyen ne change pas trop en0tre les départements de la région occitanie.

---
**Distribution de l'IPS moyen en fonction de l'académie pour les collèges en 2021**

In [515]:
# IPS moyen en fct du département
fig1 = px.histogram(dfIPSALL, x="Académie", y="IPS", color="Académie", histfunc="avg")
fig1.update_layout(title_text="Distribution de l'IPS moyen en fonction de l'académie pour les collèges en 2021")
fig1.update_yaxes(title_text="IPS")
fig1.update_xaxes(title_text="Académie")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On constate que **l'IPS moyen** de l'académie de Toulouse est supérieur de **6,3** points par rapport à celui de Montpellier.

## Croisement des données des collèges

In [516]:
# Remodification du CSV pour mise en forme (ETL)
# importing the csv files
dflabels = pd.read_csv("fr-en-occitanie-ac-montpellier-label-numerique-college.csv", sep = ";")
# Trie la colonne Label par labels (ascendant : de label 1 à label 3)
dflabels = dflabels.sort_values(by="Label")
# Transforme la colonne en string afin de pouvoir l'utiliser pour les graphiques
dflabels["Label"] = dflabels["Label"].apply(str)
dflabels["Département"] = dflabels["Département"].apply(str)

In [517]:
# Préparation des données du df des labels afin de prendre en compte que l'année 2021 
# pour matcher avec le fait que pour le df IPS on a des infos que sur 2021/2022
dflabels = dflabels[dflabels["Année"] == 2021]
dflabels

,Année,RNE,Nom établissement,Localité,Département,Label,Position
519,2021,0300945P,Collège Feuchères,Nîmes,30,1,"43.8346647285, 4.36436997917"
950,2021,0342050U,Collège Françoise Giroud,Vendres,34,1,"43.2702388366, 3.23120964337"
951,2021,0342130F,Collège Pierre Deley,Marseillan,34,1,"43.3636032312, 3.52046025283"
955,2021,0660007Z,Collège Irène Joliot Curie,Estagel,66,1,"42.7703572832, 2.69483024839"
513,2021,0110859Z,Collège Joseph Delteil,Limoux,11,1,"43.0641830596, 2.21530968734"
...,...,...,...,...,...,...,...
540,2021,0660016J,Collège la Garrigole,Perpignan,66,3,"42.6948472397, 2.86727657243"
536,2021,0480002S,Collège Henri Gamala,Le Collet-de-Dèze,48,3,"44.2458831779, 3.91986047952"
531,2021,0340119V,Collège Ferdinand Fabre,Bédarieux,34,3,"43.6109247059, 3.15462945714"
529,2021,0340070S,Collège du Jaur,Saint-Pons-de-Thomières,34,3,"43.485152687, 2.77101015912"


In [518]:
# Jointure afin de croiser les RNE du df des labels et les UAI du df des IPS 
# pour avoir des infos sur le lien entre les 2 
left = dflabels.set_index(['RNE'])
right = dfIPSM.set_index(['UAI'])

dflabels_IPS = left.join(right, lsuffix='_dflabels', rsuffix='_dfIPS')

dflabels_IPS


,Année,Nom établissement,Localité,Département_dflabels,Label,Position,Rentrée scolaire,Académie,Code du département,Département_dfIPS,Nom de l'établissment,Code INSEE de la commune,Nom de la commune,Secteur,IPS
RNE,,,,,,,,,,,,,,,
0300945P,2021,Collège Feuchères,Nîmes,30,1,"43.8346647285, 4.36436997917",2021-2022,MONTPELLIER,030,GARD,COLLEGE FEUCHERES,30189,NIMES,public,87.5
0342050U,2021,Collège Françoise Giroud,Vendres,34,1,"43.2702388366, 3.23120964337",2021-2022,MONTPELLIER,034,HERAULT,COLLEGE FRANCOISE GIROUD,34329,VENDRES,public,102.1
0342130F,2021,Collège Pierre Deley,Marseillan,34,1,"43.3636032312, 3.52046025283",2021-2022,MONTPELLIER,034,HERAULT,COLLEGE PIERRE DELEY,34150,MARSEILLAN,public,91.4
0660007Z,2021,Collège Irène Joliot Curie,Estagel,66,1,"42.7703572832, 2.69483024839",2021-2022,MONTPELLIER,066,PYRENEES-ORIENTALES,COLLEGE IRENE JOLIOT CURIE,66071,ESTAGEL,public,100.2
0110859Z,2021,Collège Joseph Delteil,Limoux,11,1,"43.0641830596, 2.21530968734",2021-2022,MONTPELLIER,011,AUDE,COLLEGE JOSEPH DELTEIL,11206,LIMOUX,public,98.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0660016J,2021,Collège la Garrigole,Perpignan,66,3,"42.6948472397, 2.86727657243",2021-2022,MONTPELLIER,066,PYRENEES-ORIENTALES,COLLEGE LA GARRIGOLE,66136,PERPIGNAN,public,74.9
0480002S,2021,Collège Henri Gamala,Le Collet-de-Dèze,48,3,"44.2458831779, 3.91986047952",2021-2022,MONTPELLIER,048,LOZERE,COLLEGE HENRI GAMALA,48051,LE COLLET DE DEZE,public,105.1
0340119V,2021,Collège Ferdinand Fabre,Bédarieux,34,3,"43.6109247059, 3.15462945714",2021-2022,MONTPELLIER,034,HERAULT,COLLEGE FERDINAND FABRE,34028,BEDARIEUX,public,94.4


*Ici on a récupérer les données en 2021 des collèges, notamment leur label et leur IPS* 

---
**Distribution de l'IPS moyen en fonction du label pour les collèges de l'académie de Montpellier en 2021**

In [519]:
# IPS cumulé en fct du label
fig1 = px.histogram(dflabels_IPS, x="Label", y="IPS", color="Label", histfunc="avg")
fig1.update_layout(title_text="Distribution de l'IPS moyen en fonction du label pour les collèges de l'académie de Montpellier en 2021")
fig1.update_yaxes(title_text="IPS")
fig1.update_xaxes(title_text="Label")
# fig1.show()

image = fig1.to_image(format='png',width=1200, height=700, scale=2)
Image(image)

On remarque que l'**IPS moyen** des collèges labéllisés **2** est **supérieur** à celui des collèges labéllisés **3** ce qui semble incohérent à premiers abords.


Ainsi pour voir s'il y'a réellement un lien entre le label et l'IPS il faudrait effectuer une **ANOVA** pour déterminer **l'effet du label sur l'IPS**.

In [520]:
# Préparation des données du df des résultats afin de prendre en compte que l'année 2021 et 2022
# pour matcher avec le fait que pour le df IPS on a des infos que sur 2021/2022
dfres22 = df[df["année"] == 2022]
# les collèges montpellier et toulouse
dfres21 = df[df["année"] == 2021]
dfres = pd.concat([dfres21,dfres22])
dfres

,année,académie,série,inscrits,inscrits filles,présents,présents filles,admis,admis filles,admis sans mention,admis mention assez bien,admis mention bien,admis mention très bien,genre,âge,présence,admissibilité,admissibilité détails,Résultats,Effectifs
333,2021,MONTPELLIER,Serie generale,28530,14455,28178,14272,25305,13116,4691,5999,7065,7550,Garçons,15 ans ou moins,Présents,Non admis,e. Non admis,non_admis_garcons,1717
334,2021,MONTPELLIER,Serie generale,28530,14455,28178,14272,25305,13116,4691,5999,7065,7550,Garçons,15 ans ou moins,Présents,Admis,d. Admis sans mention,admis_sans_mention_garcons,2514
335,2021,MONTPELLIER,Serie generale,28530,14455,28178,14272,25305,13116,4691,5999,7065,7550,Garçons,15 ans ou moins,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,3422
336,2021,MONTPELLIER,Serie professionnelle,1892,691,1794,647,1505,572,480,563,368,94,Filles,15 ans ou moins,Présents,Admis,d. Admis sans mention,admis sans mention filles,175
337,2021,MONTPELLIER,Serie professionnelle,1892,691,1794,647,1505,572,480,563,368,94,Filles,15 ans ou moins,Présents,Admis,c. Admis mention assez bien,admis mention assez bien filles,206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,2021,TOULOUSE,Serie professionnelle,1479,558,1335,500,983,370,424,330,187,42,Garçons,16 ans,Présents,Admis,b. Admis mention bien,admis_mention_bien_garcons,118
1724,2021,TOULOUSE,Serie generale,91,35,65,26,36,16,22,10,2,2,Filles,17 ans ou plus,Présents,Admis,b. Admis mention bien,admis mention bien filles,0
1725,2021,TOULOUSE,Serie generale,91,35,65,26,36,16,22,10,2,2,Garçons,17 ans ou plus,Non présents,Non présents,f. Non présents,non_presents_garcons,17
1726,2021,TOULOUSE,Serie professionnelle,164,56,135,49,91,33,42,27,15,7,Filles,17 ans ou plus,Présents,Admis,d. Admis sans mention,admis sans mention filles,17


**Import des bibliothèques pour l'ANOVA**

In [521]:
# Import des bibliothèques pour l'anova
import statsmodels.api as sa
import statsmodels.formula.api as sfa
!pip install scikit_posthocs
import scikit_posthocs as sp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Analyse de la variance**

Nous souhaitons répondre à la question suivante : y'a t'il un effet de la labélisation sur l'IPS ?

Pour cela on pose les hypothèses suivantes :

**H0** : La labélisation d'un collège à Montpellier en 2021 n'a pas d'effet sur l'IPS

**H1** : La labélisation d'un collège à  a un effet sur l'IPS

Etant donné que Le lien entre des variables en ANOVA est réciproque car les variables indépendantes et dépendantes sont liées entre elles de manière réciproque. Une variable indépendante peut affecter une variable dépendante, et vice-versa. Ainsi on pourras tout autant conclure sur l'effet de l'IPS sur la labélisation.

In [522]:
# ANOVA labels et IPS
lm = sfa.ols('IPS ~ C(Label)', data=dflabels_IPS).fit()
anova = sa.stats.anova_lm(lm)
anova

,df,sum_sq,mean_sq,F,PR(>F)
C(Label),2.0,202.007476,101.003738,0.475485,0.622412
Residual,169.0,35899.379675,212.422365,NaN,NaN


Analyse de l'anova

P_value > alpha (0,05), donc on ne rejette pas H0 et **on conclut d'une manière significative qu'il n'y a pas d'effet de la labélisation sur l'IPS pour les collèges de Montpellier en 2021** et que donc **il n'y a pas non plus d'effet de l'IPS sur la labélisation**.